# Init

In [1]:
import os
import openai
from tqdm import tqdm

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type= os.environ["OPENAI_API_TYPE"]
openai.api_version = os.environ["OPENAI_API_VERSION"]
OPENAI_DEPLOYMENT_ID = os.environ["OPENAI_DEPLOYMENT_ID"]

In [3]:
# to disable SSL verification that causes problem from my laptop
import contextlib2
import warnings
import requests
from urllib3.exceptions import InsecureRequestWarning
# Use a context manager for disabling SSL verification with 3rd party modules
old_merge_environment_settings = requests.Session.merge_environment_settings

@contextlib2.contextmanager
def no_ssl_verification():
    opened_adapters = set()

    def merge_environment_settings(self, url, proxies, stream, verify, cert):
        # Verification happens only once per connection so we need to close
        # all the opened adapters once we're done. Otherwise, the effects of
        # verify=False persist beyond the end of this context manager.
        opened_adapters.add(self.get_adapter(url))

        settings = old_merge_environment_settings(
            self, url, proxies, stream, verify, cert)
        settings['verify'] = False

        return settings

    requests.Session.merge_environment_settings = merge_environment_settings

    try:
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', InsecureRequestWarning)
            yield
    finally:
        requests.Session.merge_environment_settings = old_merge_environment_settings

        for adapter in opened_adapters:
            try:
                adapter.close()
            except:
                pass

In [8]:
def get_completion(prompt, deployment_id):
    with no_ssl_verification():
        messages = [{"role": "user", "content": prompt}]
        response = openai.ChatCompletion.create(
            deployment_id=deployment_id,
            messages=messages,
            temperature=0, 
        )
    return response.choices[0].message["content"]

# Play!

In [4]:
prompt = f"When I was 6 my sister was half my age. Now\
I’m 70 how old is my sister?"

In [9]:
get_completion(prompt,OPENAI_DEPLOYMENT_ID)


'Your sister is 67 years old.'